# RIODatasetSceneGraph class
* no use rio27!
* only deal with training!

In [72]:
import torch.utils.data as data
import os
import os.path
import copy
import json
import random

import torch
import numpy as np
import pickle
from tqdm import tqdm

import graphto3d.dataset.util as util
from graphto3d.helpers.psutil import FreeMemLinux
from graphto3d.helpers.util import normalize_box_params, denormalize_box_params, get_rotation
from graphto3d.model.atlasnet import AE_AtlasNet

os.chdir("/root/")
print(os.getcwd())

/root


In [33]:
# loading atlas model
path2atlas = "/root/graphto3d/experiments/atlasnet/model_70.pth"
saved_atlasnet_model = torch.load(path2atlas)
point_ae = AE_AtlasNet(num_points=1024, bottleneck_size=128, nb_primitives=25)
point_ae.load_state_dict(saved_atlasnet_model, strict=True)
if torch.cuda.is_available():
    point_ae = point_ae.cuda()
point_ae.eval()

AE_AtlasNet(
  (encoder): Sequential(
    (0): PointNetfeat(
      (stn): STN3d(
        (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
        (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
        (fc1): Linear(in_features=1024, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
        (fc3): Linear(in_features=256, out_features=9, bias=True)
        (relu): ReLU()
      )
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Linear(in_

## Other Functions

In [122]:
def load_points(filename, factor=1, filter_mask=False):
    point_set, _, _, mask = util.read_ply(filename)
    if filter_mask:
        point_set = point_set[np.where(mask > 0)[0], :]
    choice = np.random.choice(len(point_set), npoints * factor, replace=True)
    point_set = point_set[choice, :]
    if len(mask) > 0:
        mask.shape = (mask.shape[0], 1)  # in place reshape
        mask = mask[choice, :]
        mask = torch.from_numpy(np.array(mask, dtype=np.uint8))
    point_set = torch.from_numpy(point_set)
    return point_set, mask

In [158]:
def norm_tensor(p, params7=None, scale=False, center=True, rotation=False, scale_func='diag'):
    """ Given a set of points of an object and (optionally) a oriented 3D box, normalize points

    :param p: tensor of 3D pointset
    :param params7: bounding box parameters [W, L, H, Cx, Cy, Cy, Z]
    :param scale: boolean, if true apply scaling to pointset p according to scale_func
    :param center: boolean, if true normalize the center of points to 0,0,0
    :param rotation: boolean, if true rotate points based on the box rotation in param7
    :param scale_func: string specifying the function used for scaling. 'diag' normalizes the diagonal to length 1.
    'whl' sets each dimension to range [-1,1].
    :return: the normalized tensor of 3D pointset
    """
    if center:
        if params7 is None:
            # this is center of mass
            mean = torch.mean(p, dim=0)
        else:
            # get center from box center if available
            mean = torch.from_numpy(params7[3:6].astype("float32"))
        p -= mean.unsqueeze(0)
    if rotation and params7 is not None:
        p = (torch.from_numpy(get_rotation(-params7[-1], degree=False).astype("float32")) @ p.T).T
    if scale and params7 is not None:
        # first if needed rotate to canonical rotation
        # apply scaling
        # if needed rotate back
        if not rotation:
            p = (torch.from_numpy(get_rotation(-params7[-1], degree=False).astype("float32")) @ p.T).T
        if scale_func == 'diag':
            # OPTION 1: normalize diagonal = 1
            norm2 = np.linalg.norm(params7[:3].astype("float32"))
            p /= norm2
        elif scale_func == 'whl':
            # OPTION 2: normalize each axis by H, W, L
            norm2 = torch.from_numpy(params7[:3].astype("float32")).reshape(1, 3)
            min_p = p.min(0)[0]
            p = ((p - min_p) / norm2) * 2. - 1.  # between -1 and 1 in all directions
        elif scale_func == 'whl_after':
            norm2 = p.max(0)[0] - p.min(0)[0]
            min_p = p.min(0)[0]
            p = ((p - min_p) / norm2) * 2. - 1.  # between -1 and 1 in all directions
        else:
            raise NotImplementedError
        if not rotation:
            p = (torch.from_numpy(get_rotation(params7[-1], degree=False).astype("float32")) @ p.T).T
    return p

In [159]:
def load_semseg(json_file):
    """ Loads semantic segmentation from json file

    :param json_file: path to file
    :return: dict, that maps instance label to text semantic label
    """
    instance2label = {}
    with open(json_file, "r") as read_file:
        data = json.load(read_file)
        for segGroups in data['segGroups']:
            instance2label[segGroups["id"]] = segGroups["label"].lower()
    return instance2label

## \_\_init\_\_

* inputs when trainining

In [32]:
root = "/root/graphto3d/GT"
root_3rscan = "/root/dev/G3D/3RScan"
label_file = "labels.instances.align.annotated.ply"
npoints = 1024
split = 'train_scans'
shuffle_objs = True
use_points = False # False?? why??
use_scene_rels = True
with_changes = False
vae_baseline = False # True if 'network_type' == 'sln'
with_feats = True
large = True
atlas = point_ae
seed=False
use_splits = True
use_rio27 = False
use_canonical = True
crop_floor = False # True일 때도 고려
center_scene_to_floor = crop_floor # True일 때도 고려
recompute_feats = False

In [128]:
# vars in init function
scale_func = 'diag'
catfile = os.path.join(root, 'classes.txt')
cat = {}
data_augmentation = True
data_len = None
fm = FreeMemLinux('GB')
padding = 0.2
pass_scan_id = False
class_choice = None

### Load files

In [139]:
# relationship reading function (in the class)
def read_relationship_json(json_file, box_json_file):
    """ Reads from json files the relationship labels, objects and bounding boxes

    :param json_file: file that stores the objects and relationships
    :param box_json_file: file that stores the oriented 3D bounding box parameters
    :return: three dicts, relationships, objects and boxes and scans
    """
    rel = {}
    objs = {}
    tight_boxes = {}
    scans = []

    with open(box_json_file, "r") as read_file:
        box_data = json.load(read_file)

    with open(json_file, "r") as read_file:
        data = json.load(read_file)
        for scan in data['scans']:

            relationships = []
            for realationship in scan["relationships"]:
                realationship[2] -= 1
                relationships.append(realationship)

            # for every scan in rel json, we append the scan id
            rel[scan["scan"] + "_" + str(scan["split"])] = relationships
            scans.append(scan["scan"] + "_" + str(scan["split"]))

            objects = {}
            boxes = {}
            for k, v in scan["objects"].items():
                objects[int(k)] = v
                try:
                    boxes[int(k)] = {}
                    boxes[int(k)]['param7'] = box_data[scan["scan"]][k]["param7"]
                    boxes[int(k)]['param7'][6] = np.deg2rad(boxes[int(k)]['param7'][6])
                    if use_canonical:
                        if "direction" in box_data[scan["scan"]][k].keys():
                            boxes[int(k)]['direction'] = box_data[scan["scan"]][k]["direction"]
                        else:
                            boxes[int(k)]['direction'] = 0
                except:
                    # probably box was not saved because there were 0 points in the instance!
                    continue
            objs[scan["scan"] + "_" + str(scan["split"])] = objects
            tight_boxes[scan["scan"] + "_" + str(scan["split"])] = boxes
    return (rel, objs, tight_boxes), scans

In [140]:
def read_relationships(read_file):
    """load list of relationship labels

    :param read_file: path of relationship list txt file
    """
    relationships = []
    with open(read_file, 'r') as f:
        for line in f:
            relationship = line.rstrip().lower()
            relationships.append(relationship)
    return relationships

In [141]:
# Load class and relationships
vocab = {}
with open(os.path.join(root, 'classes.txt'), "r") as f:
    vocab['object_idx_to_name'] = f.readlines()
with open(os.path.join(root, 'relationships.txt'), "r") as f:
    vocab['pred_idx_to_name'] = f.readlines()

# Load scan_id for training
splitfile = os.path.join(root, '{}.txt'.format(split)) # split : 'train_scans'
filelist = open(splitfile, "r").read().splitlines()

print("splitfile: ", splitfile)
print("first element of filelist: ", filelist[0]) # 3rscan folder name

# assign fname according to training set or validation set
if split == 'train_scans': # training set
    splits_fname = 'relationships_train_clean' if use_splits else 'relationships_merged_train_clean'
    rel_json_file = os.path.join(root, '{}.json'.format(splits_fname))
    box_json_file = os.path.join(root, 'obj_boxes_train_refined.json')
    floor_json_file = os.path.join(root, 'floor_boxes_split_train.json')
else: # validation set
    splits_fname = 'relationships_validation_clean' if use_splits else 'relationships_merged_validation_clean'
    rel_json_file = os.path.join(root, '{}.json'.format(splits_fname))
    box_json_file = os.path.join(root, 'obj_boxes_val_refined.json')
    floor_json_file = os.path.join(root, 'floor_boxes_split_val.json')

print("rel_json_file: ", rel_json_file)
print("box_json_file: ", box_json_file)
print("floor_json_file: ", floor_json_file)

splitfile:  /root/graphto3d/GT/train_scans.txt
first element of filelist:  7272e161-a01b-20f6-8b5a-0b97efeb6545
rel_json_file:  /root/graphto3d/GT/relationships_train_clean.json
box_json_file:  /root/graphto3d/GT/obj_boxes_train_refined.json
floor_json_file:  /root/graphto3d/GT/floor_boxes_split_train.json


In [142]:
# load floor-cropped data
if crop_floor:
    with open(floor_json_file, "r") as read_file:
        floor_data = json.load(read_file)
else:
    print("crop_floor is setted to {}".format(crop_floor))

crop_floor is setted to False


In [143]:
# Load relationships, objs, bboxes
jsonlist, scans = read_relationship_json(rel_json_file, box_json_file)
relationship_json, objs_json, tight_boxes_json = jsonlist[0], jsonlist[1], jsonlist[2]

In [144]:
# results
jsonlist = [relationship_json, objs_json, tight_boxes_json]
fnamelist = ['relationship_json', 'objs_json', 'tight_boxes_json']

for i in range(len(jsonlist)):
    print("for the {} file:".format(fnamelist[i]))
    tmp = list(jsonlist[i].keys())[0]
    print("first key of {} file: {}".format(fnamelist[i], tmp))
    if str(type(jsonlist[i][tmp])) == "<class 'list'>":
        print("first value of {} file: {}".format(fnamelist[i], json.dumps(jsonlist[i][tmp][0], indent=4)))
        print()
    else:
        print("value of {} file: {}".format(fnamelist[i], json.dumps(jsonlist[i][tmp], indent=4)))
        print()

for the relationship_json file:
first key of relationship_json file: f62fd5fd-9a3f-2f44-883a-1e5cf819608e_1
first value of relationship_json file: [
    4,
    1,
    13,
    "attached to"
]

for the objs_json file:
first key of objs_json file: f62fd5fd-9a3f-2f44-883a-1e5cf819608e_1
value of objs_json file: {
    "1": "floor",
    "26": "wall",
    "25": "lamp",
    "13": "cabinet",
    "4": "wall",
    "42": "stool",
    "44": "pillow",
    "14": "cabinet",
    "15": "cabinet"
}

for the tight_boxes_json file:
first key of tight_boxes_json file: f62fd5fd-9a3f-2f44-883a-1e5cf819608e_1
value of tight_boxes_json file: {
    "1": {
        "param7": [
            8.300434079221413,
            4.541183126091166,
            0.22766995429992676,
            0.9669842840141765,
            0.13836722910367122,
            -1.4374949932098389,
            1.8938213499224684e-09
        ],
        "direction": 2
    },
    "26": {
        "param7": [
            0.3735505047342036,
          

### Set variables

* classes with categories

In [145]:
# set the classes with categories
with open(catfile, 'r') as f:
    for line in f:
        category = line.rstrip()
        cat[category] = category
if not class_choice is None:
    cat = {k: v for k, v in cat.items() if k in class_choice}

count = 0
for key in cat.keys():
    print('{}th element in cat -> "{}" : "{}"'.format(count, key, cat[key]))
    count += 1
    if count > 5: break
print("number of elements in cat: ", len(cat))
print()

# classes
classes = dict(zip(sorted(cat), range(len(cat))))
print('classes:\n', classes)

0th element in cat -> "_scene_" : "_scene_"
1th element in cat -> "armchair" : "armchair"
2th element in cat -> "backpack" : "backpack"
3th element in cat -> "bag" : "bag"
4th element in cat -> "ball" : "ball"
5th element in cat -> "bar" : "bar"
number of elements in cat:  161

classes:
 {'_scene_': 0, 'armchair': 1, 'backpack': 2, 'bag': 3, 'ball': 4, 'bar': 5, 'basin': 6, 'basket': 7, 'bath cabinet': 8, 'bathtub': 9, 'bed': 10, 'bedside table': 11, 'bench': 12, 'bidet': 13, 'bin': 14, 'blanket': 15, 'blinds': 16, 'board': 17, 'book': 18, 'books': 19, 'bookshelf': 20, 'bottle': 21, 'box': 22, 'bread': 23, 'bucket': 24, 'cabinet': 25, 'carpet': 26, 'ceiling': 27, 'chair': 28, 'cleanser': 29, 'clock': 30, 'closet': 31, 'clothes': 32, 'clothes dryer': 33, 'clutter': 34, 'coffee machine': 35, 'coffee table': 36, 'commode': 37, 'computer desk': 38, 'couch': 39, 'couch table': 40, 'counter': 41, 'cup': 42, 'cupboard': 43, 'curtain': 44, 'cushion': 45, 'cutting board': 46, 'decoration': 47, 

* points_classes_idx
    * Discard some underrepresented classes for the shape generation

In [146]:
if not large:
    points_classes = ['bed', 'chair', 'armchair', 'desk', 'door', 'floor', 'picture', 'sofa', 'couch',
                      'stool', 'table']
else: # larger set of shape classes
    points_classes = ['ball', 'basket', 'bench', 'bed', 'box', 'cabinet', 'chair', 'armchair',
                      'desk', 'door', 'floor', 'picture', 'sofa', 'couch', 'commode', 'monitor',
                      'stool', 'tv', 'table']

points_classes_idx = []
for pc in points_classes:
    if class_choice is not None:
        if pc in classes:
            points_classes_idx.append(classes[pc])
        else:
            points_classes_idx.append(0)
    else:
        if not use_rio27:
            points_classes_idx.append(classes[pc])
        else:
            points_classes_idx.append(int(vocab_rio27['rio27_name_to_idx'][pc]))

print('points_classes_idx: ', points_classes_idx)
print()

classlist = []
for idx in points_classes_idx:
    for key in classes.keys():
        if classes[key] == idx:
            classlist.append(key)
print('classes corresponding to points_classes_idx:\n', classlist)

points_classes_idx:  [4, 7, 12, 10, 22, 25, 28, 1, 48, 51, 58, 102, 132, 39, 37, 89, 136, 152, 140]

classes corresponding to points_classes_idx:
 ['ball', 'basket', 'bench', 'bed', 'box', 'cabinet', 'chair', 'armchair', 'desk', 'door', 'floor', 'picture', 'sofa', 'couch', 'commode', 'monitor', 'stool', 'tv', 'table']


## \_\_getitem\_\_

* input

In [149]:
index = 0

* scan id, split

In [152]:
scan_id = scans[index]

scan_id_no_split = scan_id.split('_')[0]
split = scan_id.split('_')[1]

print('scan_id:', scan_id)
print('scan_id_no_split:', scan_id_no_split)
print('split:', split)

# if crop_floor is true, save floor index and set the center of the scene
if crop_floor:
    scene_floor = floor_data[scan_id_no_split][split]
    floor_idx = list(scene_floor.keys())[0]
    if center_scene_to_floor:
        scene_center = np.asarray(scene_floor[floor_idx]['params7'][3:6])
    else:
        scene_center = np.array([0, 0, 0])

    min_box = np.asarray(scene_floor[floor_idx]['min_box']) - scene_center
    max_box = np.asarray(scene_floor[floor_idx]['max_box']) - scene_center

scan_id: f62fd5fd-9a3f-2f44-883a-1e5cf819608e_1
scan_id_no_split: f62fd5fd-9a3f-2f44-883a-1e5cf819608e
split: 1


### get semseg of the scene from 3RScan data
* instance2label : A dictionary loaded from 3RScan data (id : label)

In [156]:
file = os.path.join(root_3rscan, scan_id_no_split, label_file)
if os.path.exists(os.path.join(root_3rscan, scan_id_no_split, "semseg.v2.json")):
    semseg_file = os.path.join(root_3rscan, scan_id_no_split, "semseg.v2.json")
elif os.path.exists(os.path.join(root_3rscan, scan_id_no_split, "semseg.json")):
    semseg_file = os.path.join(root_3rscan, scan_id_no_split, "semseg.json")
else:
    raise FileNotFoundError("Cannot find semseg.json file.")

print(semseg_file)

/root/dev/G3D/3RScan/f62fd5fd-9a3f-2f44-883a-1e5cf819608e/semseg.v2.json


In [163]:
instance2label = load_semseg(semseg_file)
selected_instances = list(objs_json[scan_id].keys())
keys = list(instance2label.keys())

print('instance2label: ', instance2label)
print('\nobjs_json of the scene: ', objs_json[scan_id])
print('\nselected_instances: ', selected_instances)
print('\nkeys: ', keys)

if shuffle_objs:
    random.shuffle(keys)

instance2label:  {20: 'chair', 21: 'chair', 22: 'kitchen cabinet', 23: 'kitchen counter', 24: 'wall', 1: 'floor', 2: 'sofa', 3: 'sofa couch', 4: 'wall', 5: 'wall', 6: 'blinds', 7: 'blinds', 8: 'table', 9: 'wall', 11: 'cabinet', 12: 'cabinet', 13: 'cabinet', 14: 'cabinet', 15: 'cabinet', 16: 'wall', 17: 'curtain', 18: 'cabinet', 19: 'chair', 25: 'lamp', 26: 'wall', 27: 'chair', 28: 'wall', 29: 'plant', 30: 'lamp', 33: 'stool', 34: 'chair', 35: 'chair', 36: 'chair', 38: 'tv', 39: 'lamp', 40: 'curtain', 41: 'curtain', 42: 'stool', 43: 'pillow', 44: 'pillow', 45: 'pillow', 46: 'pillow', 47: 'pillow', 48: 'pillow', 49: 'pillow', 50: 'pillow', 52: 'pillow', 53: 'item', 54: 'item', 55: 'sink', 56: 'plant', 57: 'box', 58: 'book', 59: 'knife box', 60: 'pepper', 61: 'blinds', 62: 'blinds', 63: 'curtain', 64: 'lamp', 76: 'lamp'}

objs_json of the scene:  {1: 'floor', 26: 'wall', 25: 'lamp', 13: 'cabinet', 4: 'wall', 42: 'stool', 44: 'pillow', 14: 'cabinet', 15: 'cabinet'}

selected_instances:  [1

### Prepare for loading the features (path)

In [169]:
feats_in = None

# if true, expected paths to saved atlasnet features will be set here
if with_feats and path2atlas is not None:
    _, atlasname = os.path.split(path2atlas)
    atlasname = atlasname.split('.')[0]
    print('atlasname: ', atlasname)
    
    if not large:
        feats_path = os.path.join(root_3rscan, scan_id.split('_')[0],
                                 '{}_small_{}_{}.pkl'.format(atlasname,
                                                            'splits' if use_splits else 'merged',
                                                            scan_id.split('_')[1]))
    else:
        feats_path = os.path.join(root_3rscan, scan_id.split('_')[0],
                                 '{}_large_{}_{}.pkl'.format(atlasname,
                                                            'splits' if use_splits else 'merged',
                                                            scan_id.split('_')[1]))
        if crop_floor:
            feats_path = os.path.join(root_3rscan, scan_id.split('_')[0],
                                 '{}_large_{}_{}_floor.pkl'.format(atlasname,
                                                                   'splits' if use_splits else 'merged',
                                                                   scan_id.split('_')[1]))
    if recompute_feats:
        feats_path += 'tmp'

print('feats_path: ', feats_path)

atlasname:  model_70
feats_path:  /root/dev/G3D/3RScan/f62fd5fd-9a3f-2f44-883a-1e5cf819608e/model_70_large_splits_1.pkl


### Load points
* keys : keys of instance2label (id of instances)
* classes : total obj classes in 3RScan {'label' : id}
* selected_instances : object ids in the scene

In [185]:
files = {}

if (with_feats and (not os.path.exists(feats_path) or recompute_feats)) or use_points:
    if file in files: # Caching
        (points, instances) = files[file]
    else:
        points, instances, _, _ = util.read_ply(file) # gets only objectId, not a globalId

        if fm.user_free > 5:
            files[file] = (points, instances)

    if crop_floor and center_scene_to_floor:
        print("shifting points")
        points = points - scene_center.reshape(1, -1)

### Get objects from the selected list of classes of 3DSSG

In [187]:
instance2mask = {}
instance2mask[0] = 0

cat = []
tight_boxes = []

counter = 0

instances_order = []
selected_shapes = []

# print('classes: ', classes)
# print('selected_instances:', selected_instances)
# print('keys:', keys)
for key in keys: # keys : keys of instance2label
    scene_instance_id = key
    scene_instance_class = instance2label[key]
    print(scene_instance_id, scene_instance_class)
    scene_class_id = -1
    if scene_instance_class in classes:
        scene_class_id = classes[scene_instance_class]
    
    # add an dictionary element to instance2mask
    if scene_class_id != -1 and key in selected_instances:
        instance2mask[scene_instance_id] = counter + 1
        counter += 1
    else:
        instance2mask[scene_instance_id] = 0
    print('instance2mask:', instance2mask)
    
    # mask to cat
    if (scene_class_id >= 0) and (scene_instance_id > 0) and (key in selected_instances): # for the object in the scene
        # cat
        if use_canonical:
            direction = tight_boxes_json[scan_id][key]['direction']
            if direction in [-1, 0, 6]:
                # skip invalid point clouds with ambiguous direction annotation
                selected_shapes.append(False)
            else:
                selected_shapes.append(True)
        cat.append(scene_class_id)
        print('cat:', cat)
        bbox = tight_boxes_json[scan_id][key]['param7'].copy()
        if crop_floor and key in floor_data[scan_id_no_split][split].keys(): # for the floor
            bbox = floor_data[scan_id_no_split][split][key]['params7'].copy()
            bbox[6] = np.deg2rad(bbox[6]) # rotation of the floor bbox
            direction = floor_data[scan_id_no_split][split][key]['direction']
        
        # instances_order
        if use_canonical:
            if direction > 1 and direction < 5:
                # update direction-less angle with direction data (shifs it by 90 degree
                # for every added direction value)
                bbox[6] += (direction - 1) * np.deg2rad(90)
                if direction == 2 or direction == 4:
                    temp = bbox[0]
                    bbox[0] = bbox[1]
                    bbox[1] = temp
            # for other options, do not change the box
        instances_order.append(key)
        print('instances_order:', instances_order)
        
        # tight_boxes
        if not vae_baseline:
            bins = np.linspace(0, np.deg2rad(360), 24)
            angle = np.digitize(bbox[6], bins)
            bbox = normalize_box_params(bbox)
        else:
            bins = np.linspace(0, np.deg2rad(360), 24)
            bbox[6] = np.digitize(bbox[6], bins)
        tight_boxes.append(bbox)
        # print(tight_boxes)

24 wall
instance2mask: {0: 0, 24: 0}
3 sofa couch
instance2mask: {0: 0, 24: 0, 3: 0}
44 pillow
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1}
cat: [105]
instances_order: [44]
50 pillow
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0}
16 wall
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0}
13 cabinet
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2}
cat: [105, 25]
instances_order: [44, 13]
27 chair
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0}
22 kitchen cabinet
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0}
62 blinds
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0, 62: 0}
29 plant
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0, 62: 0, 29: 0}
46 pillow
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0, 62: 0, 29: 0, 46: 0}
53 item
instance2mask: {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0, 62: 0, 29: 0, 46: 0, 53: 0}
35

### If precomputed features exist, simply load them

In [217]:
if with_feats:
    if os.path.exists(feats_path):
        feats_dic = pickle.load(open(feats_path, 'rb'))
        
        feats_in = feats_dic['feats']
        print(feats_in.shape, '\n', feats_in, '\n')
        feats_order = np.asarray(feats_dic['instance_order'])
        print('feats_order: ', feats_order)
        print('instances_order: ', instances_order)
        print()
        
        ordered_feats = [] # ordered features wrt instances_order
        for inst in instances_order:
            feats_in_instance = inst == feats_order
            print(inst)
            print(feats_in_instance)
            ordered_feats.append(feats_in[:-1][feats_in_instance])
            # [:-1] → drop last index
            # [feats_in_instance] → select feature(1, 128) for only true index
        print(np.array(ordered_feats).shape)
        ordered_feats.append(np.zeros([1, feats_in.shape[1]])) # (10, 1, 128)
        print(np.array(ordered_feats).shape)
        feats_in = list(np.concatenate(ordered_feats, axis=0)) # (10, 128)
        print(np.array(feats_in).shape)

(10, 128) 
 [[0.4360339  1.1080422  0.         ... 0.         0.02795782 0.47313926]
 [0.4360339  1.1080422  0.         ... 0.         0.02795782 0.47313926]
 [0.4360339  1.1080422  0.         ... 0.         0.02795782 0.47313926]
 ...
 [0.4360339  1.1080422  0.         ... 0.         0.02795782 0.47313926]
 [0.33617944 0.2669381  0.11628396 ... 0.         0.20880023 0.22970316]
 [0.4360339  1.1080422  0.         ... 0.         0.02795782 0.47313926]] 

feats_order:  [ 4 26 14 13  1 25 44 42 15]
instances_order:  [44, 13, 1, 25, 14, 26, 42, 4, 15]

44
[False False False False False False  True False False]
13
[False False False  True False False False False False]
1
[False False False False  True False False False False]
25
[False False False False False  True False False False]
14
[False False  True False False False False False False]
26
[False  True False False False False False False False]
42
[False False False False False False False  True False]
4
[ True False False False False 

In [216]:
# indexing with boolean list
example = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
print(example.shape, '\n', example)
print()
print(example[:-1].shape, '\n', example[:-1])
print()
print(example[:-1][[True, False, False]])

(4, 2) 
 [[1 2]
 [2 3]
 [3 4]
 [4 5]]

(3, 2) 
 [[1 2]
 [2 3]
 [3 4]]

[[1 2]]


### Sampling of points from object if they are loaded (don't use at first)

In [224]:
print('instance2mask: ', instance2mask)
print('num_pointsets: ', len(cat))
if (with_feats and (not os.path.exists(feats_path) or feats_in is None)) or use_points:
    masks = np.array(list(map(lambda l: instance2mask[l] if l in instance2mask.keys() else 0, instances)),
                    dtype=np.int32)
    print('masks:', masks)
    num_pointsets = len(cat)
    obj_points = torch.zeros([num_pointsets, npoints, 3]) # (9, 1024, 3)
    
    for i in range(len(cat)):
        obj_pointset = points[np.where(masks == i + 1)[0], :]
        
        if crop_floor and vocab['object_idx_to_name'][cat[i]].split('\n')[0] == 'floor':
            filter_mask = (obj_pointset[:,0] > min_box[0]) * (obj_pointset[:,0] < max_box[0]) \
            * (obj_pointset[:,1] > min_box[1]) * (obj_pointset[:,1] < max_box[1])
            
            obj_pointset = obj_pointset[np.where(filter_mask > 0)[0], :]
            
            print(vocab['object_idx_to_name'][cat[i]].split('\n')[0], len(obj_pointset))
        if len(obj_pointset) >= npoints:
            choice = np.random.choice(len(obj_pointset), npoints, replace=False)
        else:
            choice = np.arange(len(obj_pointset))
            # use repetitions to fill some more points
            choice2 = np.random.choice(len(obj_pointset), npoints - choice.shape[0], replace=True)
            choice = np.concatenate([choice, choice2], 0)
            random.shuffle(choice)
        
        obj_pointset = obj_pointset[choice, :]
        obj_pointset = torch.from_numpy(obj_pointset.astype(np.float32))
        
        if not vae_baseline:
            obj_pointset = norm_tensor(obj_pointset, denormalize_box_params(tight_boxes[i]),
                                      scale=True, rotation=use_canonical, scale_func=scale_func)
        else:
            obj_pointset = norm_tensor(obj_pointset, np.asarray(tight_boxes[i]),
                                            scale=True, rotation=use_canonical, scale_func=scale_func)
    obj_points[i] = obj_pointset
else:
    obj_points = None

instance2mask:  {0: 0, 24: 0, 3: 0, 44: 1, 50: 0, 16: 0, 13: 2, 27: 0, 22: 0, 62: 0, 29: 0, 46: 0, 53: 0, 35: 0, 5: 0, 59: 0, 1: 3, 25: 4, 7: 0, 2: 0, 8: 0, 14: 5, 28: 0, 45: 0, 17: 0, 18: 0, 11: 0, 64: 0, 21: 0, 36: 0, 49: 0, 26: 6, 58: 0, 20: 0, 48: 0, 55: 0, 12: 0, 42: 7, 52: 0, 63: 0, 43: 0, 61: 0, 40: 0, 34: 0, 57: 0, 9: 0, 4: 8, 39: 0, 60: 0, 30: 0, 15: 9, 33: 0, 19: 0, 38: 0, 41: 0, 56: 0, 76: 0, 23: 0, 47: 0, 54: 0, 6: 0}
num_pointsets:  9


### Create Relationship Triples

In [240]:
triples = []
rel_json = relationship_json[scan_id]
# print('instance2mask: ', json.dumps(instance2mask, indent=4)) # id : counter (0 if doesn't exist in the scene)
print('rel_json: ', rel_json)

for r in rel_json: # create relationship triplets from data
    if r[0] in instance2mask.keys() and r[1] in instance2mask.keys():
        subject = instance2mask[r[0]] - 1
        object = instance2mask[r[1]] - 1
        predicate = r[2] + 1
        if subject >= 0 and object >= 0:
            triples.append([subject, predicate, object])
    else:
        continue
print('relationship triples: ', triples)

rel_json:  [[4, 1, 13, 'attached to'], [13, 1, 14, 'standing on'], [14, 4, 13, 'attached to'], [25, 1, 14, 'standing on'], [26, 1, 13, 'attached to'], [42, 1, 14, 'standing on'], [15, 4, 13, 'attached to'], [13, 25, 2, 'right'], [13, 25, 3, 'front'], [13, 42, 2, 'right'], [25, 42, 2, 'right'], [25, 42, 5, 'close by'], [42, 25, 3, 'front'], [14, 13, 7, 'bigger than'], [15, 13, 7, 'bigger than'], [15, 14, 9, 'higher than']]
relationship triples:  [[7, 14, 2], [1, 15, 2], [4, 14, 7], [3, 15, 2], [5, 14, 2], [6, 15, 2], [8, 14, 7], [1, 3, 3], [1, 4, 3], [1, 3, 6], [3, 3, 6], [3, 6, 6], [6, 4, 3], [4, 8, 1], [8, 8, 1], [8, 10, 4]]


### Add the objects from the selected list of classes of 3DSSG (cat) to the triples

In [241]:
print('cat:', cat)
print('original triples: ', triples)
if use_scene_rels:
    # add _scene_ object and _in_scene_ connections
    scene_idx = len(cat) # num of instances
    for i, ob in enumerate(cat):
        triples.append([i, 0, scene_idx])
    cat.append(0)
    # dummy scene box
    tight_boxes.append([-1, -1, -1, -1, -1, -1, -1])
print('instances added triples: ', triples)

cat: [105, 25, 58, 80, 25, 155, 136, 155, 25, 0, 0, 0, 0]
triples:  [[7, 14, 2], [1, 15, 2], [4, 14, 7], [3, 15, 2], [5, 14, 2], [6, 15, 2], [8, 14, 7], [1, 3, 3], [1, 4, 3], [1, 3, 6], [3, 3, 6], [3, 6, 6], [6, 4, 3], [4, 8, 1], [8, 8, 1], [8, 10, 4], [0, 0, 13], [1, 0, 13], [2, 0, 13], [3, 0, 13], [4, 0, 13], [5, 0, 13], [6, 0, 13], [7, 0, 13], [8, 0, 13], [9, 0, 13], [10, 0, 13], [11, 0, 13], [12, 0, 13]]


### Outputs

In [246]:
output = {}
if use_points:
    output['scene'] = points

if with_feats and (not os.path.exists(feats_path) or feats_in is None) and atlas is not None:
    pf = torch.from_numpy(np.array(list(obj_points.numpy()), dtype=np.float32)).float().cuda().transpose(1,2)
    with torch.no_grad():
        feats = atlas.encoder(pf).detach().cpu().numpy()

    feats_out = {}
    feats_out['feats'] = feats
    feats_out['instance_order'] = instances_order
    feats_in = list(feats)

    assert path2atlas is not None
    path = os.path.join(feats_path)
    if recompute_feats:
        path = path[:-3]

    pickle.dump(feats_out, open(path, 'wb'))

# prepare outputs
output['encoder'] = {}
output['encoder']['objs'] = cat
output['encoder']['triples'] = triples
output['encoder']['boxes'] = tight_boxes
if use_points:
    output['encoder']['points'] = list(obj_points.numpy())

if with_feats:
    output['encoder']['feats'] = feats_in

output['manipulate'] = {}
if not with_changes:
    output['manipulate']['type'] = 'none'
    output['decoder'] = copy.deepcopy(output['encoder'])
else: # manipulation
    if with_changes:
        output['manipulate']['type'] = ['relationship', 'addition', 'none'][
            np.random.randint(3)]  # removal is trivial - so only addition and rel change
    else:
        output['manipulate']['type'] = 'none'
    output['decoder'] = copy.deepcopy(output['encoder'])
    if output['manipulate']['type'] == 'addition':
        node_id = remove_node_and_relationship(output['encoder'])
        if node_id >= 0:
            output['manipulate']['added'] = node_id
        else:
            output['manipulate']['type'] = 'none'
    elif output['manipulate']['type'] == 'relationship':
        rel, pair, suc = modify_relship(output['decoder'])
        if suc:
            output['manipulate']['relship'] = (rel, pair)
        else:
            output['manipulate']['type'] = 'none'

# torchify
output['encoder']['objs'] = torch.from_numpy(np.array(output['encoder']['objs'], dtype=np.int64))
output['encoder']['triples'] = torch.from_numpy(np.array(output['encoder']['triples'], dtype=np.int64))
output['encoder']['boxes'] = torch.from_numpy(np.array(output['encoder']['boxes'], dtype=np.float32))
if use_points:
    output['encoder']['points'] = torch.from_numpy(np.array(output['encoder']['points'], dtype=np.float32))
if with_feats:
    output['encoder']['feats'] = torch.from_numpy(np.array(output['encoder']['feats'], dtype=np.float32))

output['decoder']['objs'] = torch.from_numpy(np.array(output['decoder']['objs'], dtype=np.int64))
output['decoder']['triples'] = torch.from_numpy(np.array(output['decoder']['triples'], dtype=np.int64))
output['decoder']['boxes'] = torch.from_numpy(np.array(output['decoder']['boxes'], dtype=np.float32))
if use_points:
    output['decoder']['points'] = torch.from_numpy(np.array(output['decoder']['points'], dtype=np.float32))
if with_feats:
    output['decoder']['feats'] = torch.from_numpy(np.array(output['decoder']['feats'], dtype=np.float32))

output['scan_id'] = scan_id_no_split
output['split_id'] = scan_id.split('_')[1]
output['instance_id'] = instances_order

In [252]:
print(output.keys())
print()
print(output)

dict_keys(['encoder', 'manipulate', 'decoder', 'scan_id', 'split_id', 'instance_id'])

{'encoder': {'objs': tensor([105,  25,  58,  80,  25, 155, 136, 155,  25,   0,   0,   0,   0,   0]), 'triples': tensor([[ 7, 14,  2],
        [ 1, 15,  2],
        [ 4, 14,  7],
        [ 3, 15,  2],
        [ 5, 14,  2],
        [ 6, 15,  2],
        [ 8, 14,  7],
        [ 1,  3,  3],
        [ 1,  4,  3],
        [ 1,  3,  6],
        [ 3,  3,  6],
        [ 3,  6,  6],
        [ 6,  4,  3],
        [ 4,  8,  1],
        [ 8,  8,  1],
        [ 8, 10,  4],
        [ 0,  0, 13],
        [ 1,  0, 13],
        [ 2,  0, 13],
        [ 3,  0, 13],
        [ 4,  0, 13],
        [ 5,  0, 13],
        [ 6,  0, 13],
        [ 7,  0, 13],
        [ 8,  0, 13],
        [ 9,  0, 13],
        [10,  0, 13],
        [11,  0, 13],
        [12,  0, 13]]), 'boxes': tensor([[-1.6146, -1.4903, -2.0074,  2.6636,  2.0038, -1.5900,  3.5464],
        [-1.9436, -1.1287, -0.1898,  5.0135,  0.4374, -2.0931, 12.3861],
      

# Collate Functions

In [253]:
def collate_fn_vaegan(batch, use_points=False):
    """
    Collate function to be used when wrapping a RIODatasetSceneGraph in a
    DataLoader. Returns a dictionary
    """

    out = {}

    out['scene_points'] = []
    out['scan_id'] = []
    out['instance_id'] = []
    out['split_id'] = []

    out['missing_nodes'] = []
    out['missing_nodes_decoder'] = []
    out['manipulated_nodes'] = []
    global_node_id = 0
    global_dec_id = 0

    for i in range(len(batch)):
        if batch[i] == -1:
            return -1
        # notice only works with single batches
        out['scan_id'].append(batch[i]['scan_id'])
        out['instance_id'].append(batch[i]['instance_id'])
        out['split_id'].append(batch[i]['split_id'])

        if batch[i]['manipulate']['type'] == 'addition':
            out['missing_nodes'].append(global_node_id + batch[i]['manipulate']['added'])
            out['missing_nodes_decoder'].append(global_dec_id + batch[i]['manipulate']['added'])
        elif batch[i]['manipulate']['type'] == 'relationship':
            rel, (sub, obj) = batch[i]['manipulate']['relship']
            out['manipulated_nodes'].append(global_dec_id + sub)
            out['manipulated_nodes'].append(global_dec_id + obj)

        if 'scene' in batch[i]:
            out['scene_points'].append(batch[i]['scene'])

        global_node_id += len(batch[i]['encoder']['objs'])
        global_dec_id += len(batch[i]['decoder']['objs'])

    for key in ['encoder', 'decoder']:
        all_objs, all_boxes, all_triples = [], [], []
        all_obj_to_scene, all_triple_to_scene = [], []
        all_points = []
        all_feats = []

        obj_offset = 0

        for i in range(len(batch)):
            if batch[i] == -1:
                print('this should not happen')
                continue
            (objs, triples, boxes) = batch[i][key]['objs'], batch[i][key]['triples'], batch[i][key]['boxes']

            if 'points' in batch[i][key]:
                all_points.append(batch[i][key]['points'])
            if 'feats' in batch[i][key]:
                all_feats.append(batch[i][key]['feats'])

            num_objs, num_triples = objs.size(0), triples.size(0)

            all_objs.append(objs)
            all_boxes.append(boxes)

            if triples.dim() > 1:
                triples = triples.clone()
                triples[:, 0] += obj_offset
                triples[:, 2] += obj_offset

                all_triples.append(triples)
                all_triple_to_scene.append(torch.LongTensor(num_triples).fill_(i))

            all_obj_to_scene.append(torch.LongTensor(num_objs).fill_(i))

            obj_offset += num_objs

        all_objs = torch.cat(all_objs)
        all_boxes = torch.cat(all_boxes)

        all_obj_to_scene = torch.cat(all_obj_to_scene)

        if len(all_triples) > 0:
            all_triples = torch.cat(all_triples)
            all_triple_to_scene = torch.cat(all_triple_to_scene)
        else:
            return -1

        outputs = {'objs': all_objs,
                   'tripltes': all_triples,
                   'boxes': all_boxes,
                   'obj_to_scene': all_obj_to_scene,
                   'tiple_to_scene': all_triple_to_scene}

        if len(all_points) > 0:
            all_points = torch.cat(all_points)
            outputs['points'] = all_points

        if len(all_feats) > 0:
            all_feats = torch.cat(all_feats)
            outputs['feats'] = all_feats
        out[key] = outputs

    return out

In [254]:
def collate_fn_vaegan_points(batch):
    """ Wrapper of the function collate_fn_vaegan to make it also return points
    """
    return collate_fn_vaegan(batch, use_points=True)